In [10]:
# tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, LeakyReLU
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score

# imports
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os

In [11]:
# Função para configurar o gerador de dados de imagem
def configure_image_data_generator():
    datagen = ImageDataGenerator(
        rescale=1/255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    return datagen

In [12]:
# Função para carregar os dados de imagem
def load_image_data(directory):
    datagen = configure_image_data_generator()
    return datagen.flow_from_directory(
        directory,
        target_size=(200, 200),
        batch_size=3,
        class_mode='categorical'
    )

In [13]:
# Carregar os dados de treinamento, validação e teste
train_dataset = load_image_data('Dataset/training/')
validation_dataset = load_image_data('Dataset/validation/')
test_dataset = load_image_data('Dataset/testing/')

Found 3213 images belonging to 3 classes.
Found 1140 images belonging to 3 classes.
Found 1052 images belonging to 3 classes.


In [14]:
# Definir a entrada da rede
input_layer = Input(shape=(200, 200, 3))

In [15]:
# Definir as camadas da rede
flatten_layer = Flatten()(input_layer)
dense_layer1 = Dense(128, activation='relu')(flatten_layer)
dense_layer2 = Dense(64, activation='relu')(dense_layer1)
output_layer = Dense(3, activation='softmax')(dense_layer2)

In [16]:
# Criar o modelo
model = Model(inputs=input_layer, 
              outputs=output_layer)

In [17]:
# Compilar o modelo
optimizer = Adadelta(learning_rate=1.0, rho=0.85)
model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [18]:
# Treinar o modelo
batch_size = 1
history = model.fit(train_dataset, 
                    validation_data=validation_dataset,
                    steps_per_epoch=len(train_dataset)//10, #batch_size,
                    validation_steps=len(validation_dataset)//batch_size,
                    epochs=10)

Epoch 1/10
107/107 [==============================] - 123s 1s/step - loss: 6.6769 - accuracy: 0.3551 - val_loss: 1.5556 - val_accuracy: 0.4482
Epoch 2/10
107/107 [==============================] - 126s 1s/step - loss: 2.6924 - accuracy: 0.4081 - val_loss: 2.1888 - val_accuracy: 0.3412
Epoch 3/10
107/107 [==============================] - 136s 1s/step - loss: 1.9012 - accuracy: 0.4112 - val_loss: 1.1617 - val_accuracy: 0.3833
Epoch 4/10
107/107 [==============================] - 134s 1s/step - loss: 1.5308 - accuracy: 0.4268 - val_loss: 1.2471 - val_accuracy: 0.3175
Epoch 5/10
107/107 [==============================] - 67s 627ms/step - loss: 1.5557 - accuracy: 0.3894 - val_loss: 1.3067 - val_accuracy: 0.4763
Epoch 6/10
107/107 [==============================] - 47s 445ms/step - loss: 1.3113 - accuracy: 0.4393 - val_loss: 1.0934 - val_accuracy: 0.3114
Epoch 7/10
107/107 [==============================] - 53s 494ms/step - loss: 1.1303 - accuracy: 0.3427 - val_loss: 1.0982 - val_accuracy: 

In [19]:
# Avaliar o modelo no conjunto de teste
y_true = test_dataset.classes
y_pred = model.predict(test_dataset)
y_pred_classes = np.argmax(y_pred, axis=1)

351/351 [==============================] - 39s 111ms/step


In [20]:
# Calcular métricas
confusion_mtx = confusion_matrix(y_true, y_pred_classes)
accuracy = accuracy_score(y_true, y_pred_classes)
classification_rep = classification_report(y_true, y_pred_classes)
roc_auc = roc_auc_score(y_true, y_pred, multi_class='ovr')

C:\Users\luist\anaconda3\envs\IC_project\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luist\anaconda3\envs\IC_project\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luist\anaconda3\envs\IC_project\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# Imprimir os resultados
print("Matriz de Confusão:")
print(confusion_mtx)
print("\nAcurácia:", accuracy)
print("\nClassification Report:")
print(classification_rep)
print("\nAUC-ROC:", roc_auc)

Matriz de Confusão:
[[354   0   6]
 [361   0   7]
 [322   0   2]]

Acurácia: 0.33840304182509506

Classification Report:
              precision    recall  f1-score   support

           0       0.34      0.98      0.51       360
           1       0.00      0.00      0.00       368
           2       0.13      0.01      0.01       324

    accuracy                           0.34      1052
   macro avg       0.16      0.33      0.17      1052
weighted avg       0.16      0.34      0.18      1052


AUC-ROC: 0.4971443231227512
